In [9]:
!pip install pydub
!pip install SpeechRecognition
!pip install pytube


Youtube section:

In [24]:
from pytube import YouTube

# Replace 'VIDEO_URL' with the actual URL of the YouTube video
video_url = 'https://www.youtube.com/watch?v=GwT6gGMRr9s&ab_channel=FryingPan'
youtube = YouTube(video_url)
video_stream = youtube.streams.filter(only_audio=True).first()
downloaded_file_path = video_stream.download('/content/videos')






In [25]:
import os

# Get the name of the file
downloaded_file_name = os.path.basename(downloaded_file_path)



In [26]:
# Specify the new filename (without extension)
new_filename = 'video.mp4'

# Download the video and rename it
downloaded_file_path = video_stream.download(output_path='/content/videos', filename=new_filename)


In [28]:
from pydub import AudioSegment

video_path = "/content/videos/video.mp4"
audio = AudioSegment.from_file(video_path, format="mp4")
audio.export("/content/videos/video.wav", format="wav")

<_io.BufferedRandom name='/content/videos/video.wav'>

In [31]:
!pip install SpeechRecognition pydub

import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Initialize recognizer class (for recognizing the speech)
r = sr.Recognizer()

def transcribe_audio(file_path):
    # load audio to pydub
    audio = AudioSegment.from_wav(file_path)
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(audio,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = audio.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    text = ''
    for i, chunk in enumerate(chunks):
        # Process each chunk
        try:
            # using google speech recognition
            print(f"Processing chunk {i+1}/{len(chunks)}")
            # create a new file "chunk_i.wav" to save that chunk
            chunk.export("./chunk{0}.wav".format(i), bitrate ='192k', format ="wav")
            # recognize the chunk
            with sr.AudioFile("./chunk{0}.wav".format(i)) as source:
                audio_listened = r.record(source)
                # try converting it to text
                chunk_text = r.recognize_google(audio_listened)
                text += ' ' + chunk_text
        except sr.UnknownValueError as e:
            print("Error:", str(e))
        except Exception as e:
            print("Error:", str(e))
    return text

print(transcribe_audio('/content/videos/video.wav'))

Processing chunk 1/29
Processing chunk 2/29
Processing chunk 3/29
Processing chunk 4/29
Processing chunk 5/29
Processing chunk 6/29
Processing chunk 7/29
Error: 
Processing chunk 8/29
Processing chunk 9/29
Processing chunk 10/29
Processing chunk 11/29
Processing chunk 12/29
Processing chunk 13/29
Processing chunk 14/29
Processing chunk 15/29
Processing chunk 16/29
Error: 
Processing chunk 17/29
Processing chunk 18/29
Processing chunk 19/29
Processing chunk 20/29
Error: 
Processing chunk 21/29
Processing chunk 22/29
Processing chunk 23/29
Error: 
Processing chunk 24/29
Error: 
Processing chunk 25/29
Processing chunk 26/29
Processing chunk 27/29
Processing chunk 28/29
Processing chunk 29/29
Error: 
 Christmas Eve December 24th 2019 I bought this course on udemy called python for financial analysis at this time I just finished my first semester of University and was actually a business major my plan was to finish the whole course during the winter break it was obviously possible since it 

In [33]:
# Save the transcribed text to a .txt file
with open('transcribed_text.txt', 'w') as f:
    f.write(transcribe_audio('/content/videos/video.wav'))

Processing chunk 1/29
Processing chunk 2/29
Processing chunk 3/29
Processing chunk 4/29
Processing chunk 5/29
Processing chunk 6/29
Processing chunk 7/29
Error: 
Processing chunk 8/29
Processing chunk 9/29
Processing chunk 10/29
Processing chunk 11/29
Processing chunk 12/29
Processing chunk 13/29
Processing chunk 14/29
Processing chunk 15/29
Processing chunk 16/29
Error: 
Processing chunk 17/29
Processing chunk 18/29
Processing chunk 19/29
Processing chunk 20/29
Error: 
Processing chunk 21/29
Processing chunk 22/29
Processing chunk 23/29
Error: 
Processing chunk 24/29
Error: 
Processing chunk 25/29
Processing chunk 26/29
Processing chunk 27/29
Processing chunk 28/29
Processing chunk 29/29
Error: 


Summarization section:

In [34]:
!pip install transformers


In [35]:
from transformers import BartTokenizer, BartForConditionalGeneration


In [36]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-xsum')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-xsum')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [37]:
with open('/content/transcribed_text.txt', 'r') as file:
    text = file.read()


In [38]:
inputs = tokenizer([text], max_length=2048, return_tensors='pt')
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=True)
summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [40]:
print(text)


 Christmas Eve December 24th 2019 I bought this course on udemy called python for financial analysis at this time I just finished my first semester of University and was actually a business major my plan was to finish the whole course during the winter break it was obviously possible since it was only 16.5 hours and it had two whole weeks so I wanted to come back from the break smarter than everyone else while they're watching Netflix and Anime January 6th 2019 winter break is over and it's the first day of school I only completed 5 hours of the course and spent the rest of my time watching Netflix and Anime I also watched a bunch of tech videos during this time and learned that you can make up to 200 k a year as a new grad software engineer at a big technology company I lost my wife my son my family I lost my job a good career over at Facebook if I just practice legal to prepare for interviews and learn how to reverse a link to list a programming question where you have to write code 

In [39]:
print(summary)


['This is the story of how I got into computer science and how I ended up becoming a software engineer.']
